## Combining DEA with external sources of data.

#### As you have probably noticed, satellite images often contain clouds. Detecting and filtering these clouds is a non-trivial task. Clouds appear in all sorts of shapes, thickness and heights and cause side effects, such as shadows, which are often quite challenging to detect.

#### In this example we are going to use precipitation data available online from the Bureau of Meteorology data services to filter cloudy days. Precipitation needs clouds and we are going to filter out dates with recorded precipitation from our DEA satellite stack. This is not always the case as daily precipitation represent the accumulation in 24 hours and it can happen that there is a clear sky at the time the satellite image is taken, but take this just as an exercise...

#### Let's start by requesting a stack of Sentinel 2 data for the area around Cairns (QLD)

In [ ]:
%matplotlib inline

import xarray as xr
from datacube import Datacube
import matplotlib.pyplot as plt

# Connect to datacube
dc = Datacube(app='DEA_BoM')

query = {
        'lat': (-16.90,-16.96),
        'lon': (145.73,145.79),
        'output_crs': 'EPSG:4326',
        'resolution': (-0.0003, 0.0003),
        'time': ('2019-01-01', '2019-03-01'),
        'measurements': ['nbart_red', 'nbart_green', 'nbart_blue']
        }

cairns = dc.load(product='s2a_ard_granule', group_by='solar_day', **query)

cairns

#### Let's visualise the first image of the stack, captured on the 4th of Jan 2019

In [ ]:
cairns[['nbart_red', 'nbart_green', 'nbart_blue']].isel(time=0).to_array().plot.imshow(robust=True, figsize=(8,8))

#### The Bureau of Meteorology offers historical daily precipitation information on its "Australian Landscape Water Balance" [website](http://www.bom.gov.au/water/landscape/)

#### Daily precipitation grids are produced by the Bureau from approximately 6,500 rain gauge stations and interpolated to a 0.05 degree (approximately 5 x 5 km) national grid.

#### These data can be downloaded but the servers where it is stored also offer a web service that can be accessed using XArray. Notice that here we are passing an http address to the `open_dataset` function instead of a path to a local file.

In [ ]:
prec = xr.open_dataset("data/rain_day_2019.nc")
prec

#### At this point we have not requested any data from the BoM server, we have just read the headers information to fill out the XArray `Dataset` object. The `prec` object does not contain the gridded precipitation data yet, that is why this operation runs very fast.

#### Now we are going to narrow down our selection using the extents defined in the DEA `cairns` Dataset. First we perform the selection spatially and then temporally. Notice the `method=nearest` parameter in the temporal selection, which we have to use to match closest dates as each Dataset contains different dates.

In [ ]:
# First we perform a spatial selection using the extents of sentinel Dataset
prec = prec.sel(latitude=slice(cairns.latitude[0],cairns.latitude[-1]), longitude=slice(cairns.longitude[0],cairns.longitude[-1]))

# Then we select the dates that intersect with the sentinel Dataset
prec = prec.sel(time=cairns.time, method='nearest')

prec

#### Due to the difference in spatial resolutions on both datasets the resulting precipitation Dataset contains just 1 pixel spatially. We can interrogate the contents of this Dataset by plotting the contents of the `rain_day` variable. Notice that we are now for the first time going to request the array data to the server, but by now we have reduced the amount of data to 12 pixels, so it is a very light and fast operation.

In [ ]:
prec.rain_day.plot()

#### Let's define the threshold of 1 mm to consider what is a rainy day and use this threshold to classify our days as rainy. Notice how we use the Numpy `flatten()` function to get rid of the extra dimensions of the data and get a 1-dimensional flat array.

In [ ]:
rainy = (prec.rain_day.values.flatten() > 1)

rainy

#### Now let's have a quick look at the sequence of satellite images of Cairns.

In [ ]:
cairns.nbart_blue.plot(col='time', col_wrap=5)

#### There are some quite cloudy images that we might be able to detect using the precipitation data. Let's repeat this plot again using the BoM data to filter out the rainy days:

In [ ]:
cairns.isel(time=~rainy).nbart_blue.plot(col='time', col_wrap=5)

#### Not bad! There are still some cloudy dry days in Cairns but we've removed most of the bad images.

#### Cairns has a tropical monsoon climate, which means that most of its precipitation happens in the summer season and it's due to convective storms. Melbourne, on the other hand use to have precipitation associated to frontal systems that come western low pressure systems. 

#### Exercise 7.1: Could you try to assess how this methodology would works in Melbourne?

> Note: Use some period in 2019 so you can use the same file for the precipitation